# 02: Performance Metrics

Calculate and analyze performance metrics for VARBX and benchmarks.


In [1]:
import sys
from pathlib import Path

current_dir = Path.cwd()

# Check if we're in notebooks/ directory or at project root
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'config.yml').exists():
    # We're already at project root
    project_root = current_dir
else:
    # Try to find project root by looking for config.yml
    project_root = current_dir
    while project_root != project_root.parent:
        if (project_root / 'config.yml').exists():
            break
        project_root = project_root.parent

sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from src.analytics.performance_metrics import (
    calculate_all_metrics,
    calculate_rolling_cagr,
    calculate_rolling_sharpe,
)
from src.utils.paths import get_data_interim_path


In [2]:
# Load cleaned data
interim_path = get_data_interim_path()
returns_df = pd.read_csv(interim_path / "returns_merged.csv", parse_dates=["date"])
returns_df = returns_df.set_index("date")

# Extract return series
varbx_returns = returns_df["return_varbx"]
sp500_returns = returns_df["return_sp500"]
hfri_ed_returns = returns_df["return_hfri_ed"]


## Calculate Performance Metrics


In [ ]:
# Calculate all metrics for each series
# VARBX with HFRI ED as benchmark (strategy benchmark)
varbx_metrics_hfri = calculate_all_metrics(
    varbx_returns,
    benchmark_returns=hfri_ed_returns
)

# VARBX with SP500 as benchmark (market benchmark)
# Align to common dates
common_dates_varbx_sp500 = varbx_returns.index.intersection(sp500_returns.index)
varbx_aligned_for_sp500 = varbx_returns.loc[common_dates_varbx_sp500]
sp500_for_varbx_benchmark = sp500_returns.loc[common_dates_varbx_sp500]

varbx_metrics_sp500 = calculate_all_metrics(
    varbx_aligned_for_sp500,
    benchmark_returns=sp500_for_varbx_benchmark
)

# SP500 with itself as benchmark (alpha=0, beta=1, avoids NaNs)
sp500_metrics = calculate_all_metrics(
    sp500_returns,
    benchmark_returns=sp500_returns
)

# HFRI ED with SP500 as benchmark (for alpha/beta calculation)
# Align to common dates
common_dates_hfri_sp500 = hfri_ed_returns.index.intersection(sp500_returns.index)
hfri_ed_aligned_for_metrics = hfri_ed_returns.loc[common_dates_hfri_sp500]
sp500_for_hfri_benchmark = sp500_returns.loc[common_dates_hfri_sp500]

hfri_ed_metrics = calculate_all_metrics(
    hfri_ed_aligned_for_metrics,
    benchmark_returns=sp500_for_hfri_benchmark
)

# Create comparison table
metrics_df = pd.DataFrame({
    "VARBX (HFRI-benchmark)": varbx_metrics_hfri,
    "VARBX (SP500-benchmark)": varbx_metrics_sp500,
    "S&P 500": sp500_metrics,
    "HFRI ED": hfri_ed_metrics
})

print("Performance Metrics Comparison")
print(metrics_df.round(4))


Performance Metrics Comparison
                                     VARBX  S&P 500  HFRI ED
cagr                                0.0660   0.1543   0.0645
total_return                        0.4521   1.3098   0.4401
cumulative_return                   0.4521   1.3098   0.4401
compound_monthly_return             0.0053   0.0120   0.0052
annualized_return                   0.0660   0.1543   0.0645
average_monthly_return              0.0054   0.0133   0.0056
average_monthly_gain                0.0075   0.0427   0.0134
average_monthly_loss               -0.0057  -0.0431  -0.0132
pct_up_months                       0.8429   0.6571   0.6857
pct_down_months                     0.1571   0.3429   0.2857
highest_monthly_performance         0.0593   0.1270   0.0484
lowest_monthly_performance         -0.0210  -0.1249  -0.0958
max_drawdown                       -0.0210  -0.2393  -0.0958
skewness                            2.6261  -0.4318  -2.1646
kurtosis                           10.6874   0.1104  1

## Rolling Metrics


## HFRI ED vs S&P 500 Comparison


In [ ]:
# Calculate metrics for HFRI ED vs S&P 500 comparison
# Align the series to common dates
common_dates = hfri_ed_returns.index.intersection(sp500_returns.index)
hfri_ed_aligned = hfri_ed_returns.loc[common_dates]
sp500_aligned = sp500_returns.loc[common_dates]

# Calculate metrics for the aligned period
# HFRI ED with SP500 as benchmark
hfri_ed_aligned_metrics = calculate_all_metrics(
    hfri_ed_aligned,
    benchmark_returns=sp500_aligned
)

# SP500 with itself as benchmark (alpha=0, beta=1)
sp500_aligned_metrics = calculate_all_metrics(
    sp500_aligned,
    benchmark_returns=sp500_aligned
)

# Create comparison table for HFRI ED vs S&P 500
hfri_sp500_comparison = pd.DataFrame({
    "HFRI ED": hfri_ed_aligned_metrics,
    "S&P 500": sp500_aligned_metrics
})

print("HFRI ED vs S&P 500 Performance Metrics Comparison")
print(f"Common period: {common_dates.min()} to {common_dates.max()}")
print(f"Number of observations: {len(common_dates)}")
print(hfri_sp500_comparison.round(4))


HFRI ED vs S&P 500 Performance Metrics Comparison
Common period: 2020-02-29 00:00:00 to 2025-11-30 00:00:00
Number of observations: 70
                                   HFRI ED  S&P 500
cagr                                0.0645   0.1543
total_return                        0.4401   1.3098
cumulative_return                   0.4401   1.3098
compound_monthly_return             0.0052   0.0120
annualized_return                   0.0645   0.1543
average_monthly_return              0.0056   0.0133
average_monthly_gain                0.0134   0.0427
average_monthly_loss               -0.0132  -0.0431
pct_up_months                       0.6857   0.6571
pct_down_months                     0.2857   0.3429
highest_monthly_performance         0.0484   0.1270
lowest_monthly_performance         -0.0958  -0.1249
max_drawdown                       -0.0958  -0.2393
skewness                           -2.1646  -0.4318
kurtosis                           12.5197   0.1104
standard_deviation               

In [5]:
# Calculate rolling metrics
window = 12  # 12-month rolling window
rolling_cagr = calculate_rolling_cagr(varbx_returns, window=window)
rolling_sharpe = calculate_rolling_sharpe(varbx_returns, window=window)

print(f"Rolling {window}-month metrics calculated")


Rolling 12-month metrics calculated
